In [7]:
# imports
import pandas as pd
import tiktoken
import os

from openai.embeddings_utils import get_embedding

In [8]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

In [12]:
# load & inspect dataset
input_datapath = "real-talk-princeton_10000.csv"  # to save space, we provide a pre-filtered dataset
df = pd.read_csv(input_datapath, index_col=0)
df = df[["Question", "Answer"]]
df = df.dropna()
df["combined"] = (
    "Question: " + df.Question.str.strip() + "; Answer: " + df.Answer.str.strip()
)
df.head(2)

,Question,Answer,combined
0,any idea when they’re updating canvas for spri...,Response from Chewy:Whenever the Professor dec...,Question: any idea when they’re updating canva...
1,I love you RTP,love you more,Question: I love you RTP; Answer: love you more


In [13]:
# subsample to 1k most recent reviews and remove samples that are too long
top_n = 10000
# df = df.sort_values("Time").tail(top_n * 2)  # first cut to first 2k entries, assuming less than half will be filtered out
# df.drop("Time", axis=1, inplace=True)

encoding = tiktoken.get_encoding(embedding_encoding)

# omit reviews that are too long to embed
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens].tail(top_n)
len(df)

9940

In [14]:
# Ensure you have your API key set in your environment per the README: https://github.com/openai/openai-python#usage

# This may take a few minutes
df["embedding"] = df.combined.apply(lambda x: get_embedding(x, engine=embedding_model))
df.to_csv("real-talk-princeton_with_embeddings_10000.csv")